# DATA EDA

### Imports

In [1]:
#Imports
import pandas as pd
import numpy as np

### Dataframes Imports 

In [2]:
#importing 1st transactions
transactions_0_df= pd.read_csv("raw_data/transactions.csv", nrows=500000)
#Importing Transactions data
transactions_df= pd.read_csv("raw_data/transactions_v2.csv",nrows=500000)
#Importing user logs data
user_logs_df = pd.read_csv("raw_data/user_logs_v2.csv",nrows=500000)
#importing members data
members_df = pd.read_csv("raw_data/members_v3.csv",nrows=500000)
# import first training dataset
train_0_df= pd.read_csv("raw_data/train.csv",nrows=500000)
# import training dataset
train_df = pd.read_csv("raw_data/train_v2.csv",nrows=500000)

# importing sample data to test 
sample_df = pd.read_csv("raw_data/sample_submission_v2.csv", nrows=500000)



### Building trendline

In [3]:
#concat vertically
transactions_df_concat = pd.concat([transactions_0_df, transactions_df], axis=0)

#drop duplicates
transactions_df_global = transactions_df_concat.drop_duplicates(keep="first")

In [4]:
#concat vertically
train_df_concat = pd.concat([train_0_df, train_df], axis=0)
train_df_concat

#drop duplicates
train_df_global = train_df_concat.drop_duplicates(keep="first")
sample_df_global = sample_df.drop_duplicates(keep="first")


In [5]:
#merge with transaction data
transactions_train_df = pd.merge(train_df_global, transactions_df_global, on='msno', how='left')
transactions_train_df

trans_sample = pd.merge(sample_df_global, transactions_df_global,on='msno', how='left')
trans_sample

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0,41.0,30.0,99.0,99.0,1.0,20151018.0,20151118.0,0.0
1,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0,41.0,30.0,99.0,99.0,1.0,20151218.0,20160118.0,0.0
2,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0,34.0,0.0,0.0,149.0,1.0,20150731.0,20150831.0,0.0
3,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
592741,VDobC8SBvwMKFZXqjjcyo1vjCVhnFCOouaTGJW0n31Y=,0,41.0,30.0,129.0,129.0,1.0,20170315.0,20170416.0,0.0
592742,237CmmUf2b3zQhuT8uaRnCkmhtQ6AIWcoGXVz9urRWg=,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
592743,B+88hLYHJzVgX2wM7D7VCshoMI3DzLAmkgb+WksauH8=,0,29.0,30.0,180.0,180.0,1.0,20170303.0,20170403.0,0.0
592744,fjFRAhH8/g0EhlwOIoFYARGw+KT0igNdTmS4Ums+XgQ=,0,38.0,30.0,149.0,149.0,0.0,20170304.0,20170403.0,0.0


In [6]:
#convert to datetime
trans_sample.loc[:,'transaction_date'] = pd.to_datetime(trans_sample['transaction_date'], format='%Y%m%d')
trans_sample.loc[:,'membership_expire_date'] = pd.to_datetime(trans_sample['membership_expire_date'], format='%Y%m%d')

In [7]:
# Sort the DataFrame by 'msno' and 'transaction_date'
trans_sample = trans_sample.sort_values(by=['msno', 'transaction_date'], ascending=[True, True],)

In [8]:
#compute previous period churns
trans_sample['transaction_date_-1'] = trans_sample.groupby('msno')['transaction_date'].shift(+1)
trans_sample['membership_expire_date_-1'] = trans_sample.groupby('msno')['membership_expire_date'].shift(+1)
trans_sample['period_0'] = trans_sample['membership_expire_date_-1']-trans_sample['transaction_date']
trans_sample['period_0_churn'] = trans_sample['period_0'].apply(lambda x: 1 if x<=pd.Timedelta(days=-30) else 0)

trans_sample['transaction_date_-2'] = trans_sample.groupby('msno')['transaction_date'].shift(+2)
trans_sample['membership_expire_date_-2'] =trans_sample.groupby('msno')['membership_expire_date'].shift(+2)
trans_sample['period_-1'] = trans_sample['membership_expire_date_-2']-trans_sample['transaction_date_-1']
trans_sample['period_-1_churn'] = trans_sample['period_-1'].apply(lambda x: 1 if x<=pd.Timedelta(days=-30) else 0)


trans_sample['transaction_date_-3'] = trans_sample.groupby('msno')['transaction_date'].shift(+3)
trans_sample['membership_expire_date_-3'] = trans_sample.groupby('msno')['membership_expire_date'].shift(+3)
trans_sample['period_-2'] = trans_sample['membership_expire_date_-3']-trans_sample['transaction_date_-2']
trans_sample['period_-2_churn'] = trans_sample['period_-2'].apply(lambda x: 1 if x<=pd.Timedelta(days=-30) else 0)


trans_sample['transaction_date_-4'] = trans_sample.groupby('msno')['transaction_date'].shift(+4)
trans_sample['membership_expire_date_-4'] = trans_sample.groupby('msno')['membership_expire_date'].shift(+4)
trans_sample['period_-3'] = trans_sample['membership_expire_date_-4']-trans_sample['transaction_date_-3']
trans_sample['period_-3_churn'] = trans_sample['period_-3'].apply(lambda x: 1 if x<=pd.Timedelta(days=-30) else 0)


trans_sample['transaction_date_-5'] = trans_sample.groupby('msno')['transaction_date'].shift(+5)
trans_sample['membership_expire_date_-5'] = trans_sample.groupby('msno')['membership_expire_date'].shift(+5)
trans_sample['period_-4'] = trans_sample['membership_expire_date_-5']-trans_sample['transaction_date_-4']
trans_sample['period_-4_churn'] = trans_sample['period_-4'].apply(lambda x: 1 if x<=pd.Timedelta(days=-30) else 0)


trans_sample['transaction_date_-6'] = trans_sample.groupby('msno')['transaction_date'].shift(+6)
trans_sample['membership_expire_date_-6'] = trans_sample.groupby('msno')['membership_expire_date'].shift(+6)
trans_sample['period_-5'] = trans_sample['membership_expire_date_-6']-trans_sample['transaction_date_-5']
trans_sample['period_-5_churn'] = trans_sample['period_-5'].apply(lambda x: 1 if x<=pd.Timedelta(days=-30) else 0)



In [ ]:
print(trans_sample.isnull().sum())
trans_sample

In [ ]:
# remove null rows from transaction date
trans_train_df = trans_sample.dropna(subset=['transaction_date'])

In [ ]:
#only keeping rows with the latest transactions 

latest_transaction_indexes = trans_train_df.groupby('msno')['transaction_date'].idxmax()
latest_transactions_with_churn = trans_train_df.loc[latest_transaction_indexes]


In [ ]:
# removing unnecessary columns
trendline_df = latest_transactions_with_churn.drop(axis=1, columns=[
    'transaction_date_-1',
    'membership_expire_date_-1',
    'period_0',
    'transaction_date_-2',
    'membership_expire_date_-2',
    'period_-1',
    'transaction_date_-3',
    'membership_expire_date_-3',
    'period_-2',
    'transaction_date_-4',
    'membership_expire_date_-4',
    'period_-3',
    'transaction_date_-5',
    'membership_expire_date_-5',
    'period_-4',
    'transaction_date_-6',
    'membership_expire_date_-6',
    'period_-5',
    'is_churn',
    'payment_method_id',
    'payment_plan_days',
    'plan_list_price',
    'actual_amount_paid',
    'is_auto_renew',
    'transaction_date',
    'membership_expire_date',
    'is_cancel'
])
                                                                        

### Transactions Data

In [13]:
###### converting transaction_date and membership_date to datetime object
transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'], format='%Y%m%d')
transactions_df['membership_expire_date'] = pd.to_datetime(transactions_df['membership_expire_date'], format='%Y%m%d')

# removing duplicates and leaving only the latest transaction date
transactions_df_lt = transactions_df.loc[transactions_df.groupby('msno').transaction_date.idxmax()]

# creating remaining plan duration column
transactions_df_lt['remaining_plan_duration'] = transactions_df_lt['membership_expire_date'] - transactions_df_lt['transaction_date']

# creating discount column
transactions_df_lt['is_discount'] = transactions_df_lt.apply(lambda x: '0' if (x['actual_amount_paid'] -x['plan_list_price'])>=0 else '1', axis=1)

### User logs Data

In [14]:
# Converting date into datetime object
user_logs_df['date'] = pd.to_datetime(user_logs_df['date'], format='%Y%m%d')

#create new dataframe from transactions table with only msno and latest transaction date
latest_transactions_per_msno = transactions_df_lt[['msno', 'transaction_date']]

#Merge user logs with latest transaction date
merged_df = user_logs_df.merge(latest_transactions_per_msno, on='msno', how='left')

# drop msno's where transaction date is NaN
merged_df = merged_df.dropna(subset=['transaction_date'])

#removing rows where the user log data is before the last transaction date
user_logs_atd= merged_df.loc[merged_df['date']>=merged_df['transaction_date']]

# removing data column
user_logs_atd = user_logs_atd.drop(columns=['date','transaction_date'])

#groupby msno and summing all values
user_logs_atd = user_logs_atd.groupby('msno').sum()

### Merging

In [15]:
# Merging transactions, user logs and members data to Train dataframe
train_df = train_df.merge(transactions_df_lt, on='msno', how='left')
train_df = train_df.merge(user_logs_atd, on='msno', how='left')
train_df = train_df.merge(members_df, on='msno', how='left')
train_df = train_df.merge(trendline_df, on='msno', how='left')

### Feature engineering

In [16]:
# Drop Gender column
train_df = train_df.drop(['gender'],axis=1)

In [17]:
# Drop NaN
train_df = train_df.dropna()

In [18]:
# Changing bd outliers to NaN
train_df['bd'] = train_df['bd'].apply(lambda x: np.nan if x <14 or x > 75 else x)

In [19]:
# Converting remaining plan duration to int
train_df['remaining_plan_duration'] = train_df['remaining_plan_duration'].dt.days

In [20]:
# Converting total seconds to hours
train_df['total_secs']= round(train_df['total_secs']/3600,2)

In [21]:
# Creating average usage (hours) per day from latest transaction
train_df['usage_from_ltd'] = round(train_df['total_secs']/train_df['remaining_plan_duration'],2)
# replacing Nan with 0 
train_df['usage_from_ltd'].replace([np.nan], 0, inplace=True)

In [22]:
# converting registration time to datetime object
train_df['registration_init_time'] = pd.to_datetime(train_df['registration_init_time'], format='%Y%m%d')

In [23]:
# creating discount percentage column
train_df['discount_percentage'] = round((train_df['plan_list_price'] - train_df['actual_amount_paid'])/train_df['plan_list_price'],2)

train_df['discount_percentage'].replace([np.nan,-0.01,0.01], 0, inplace=True)

### Cyclic Encoding

In [24]:
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [25]:
# cyclic encoding transaction date
#year
train_df['last_transaction_year'] = train_df['transaction_date'].dt.year
train_df

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,...,registration_init_time,period_0_churn,period_-1_churn,period_-2_churn,period_-3_churn,period_-4_churn,period_-5_churn,usage_from_ltd,discount_percentage,last_transaction_year
40,WpYNkiRMP3vgJRyq4TQzn7y8O0I7RWtXLXUorHClllg=,1,39.0,30.0,149.0,149.0,1.0,2017-02-28,2017-04-06,0.0,...,2014-10-30,1.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,2017
41,2OaV7/7vyAA2FrCkV4qXZEZYg5IzVyFNkQnQorgQ440=,1,41.0,30.0,149.0,149.0,1.0,2017-02-28,2017-06-08,0.0,...,2014-11-03,1.0,0.0,1.0,0.0,0.0,0.0,0.03,0.0,2017
42,VCZm6WJqkwDhmConNXKrPly5XJWE26zjE7FL9tlSqww=,1,32.0,360.0,1200.0,1200.0,0.0,2017-03-06,2018-02-26,0.0,...,2014-11-12,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,2017
195,2oFsm9EaY6lOTH0Wd+2vhbPQg2TIZ6XMdVcVlkCv//0=,1,32.0,180.0,536.0,536.0,0.0,2017-03-09,2017-09-05,0.0,...,2016-11-19,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,2017
295,8WxrF1f/6pUZyXDEvhN7CRkCzNxg0v2H2KMDL8r62aQ=,1,41.0,30.0,129.0,129.0,1.0,2017-01-31,2017-07-21,0.0,...,2010-07-30,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499291,HGlRYxOnzuhFKBbH+pgPwzraWJ+XL1ezA9yjW9JZefY=,0,41.0,30.0,99.0,99.0,1.0,2016-02-17,2018-03-17,0.0,...,2012-12-21,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,2016
499596,Xq+6sb4KXycYSJYU7y3Kd1qaqG0BPnMcxccHB3/cvzc=,0,40.0,30.0,149.0,149.0,1.0,2017-03-14,2017-04-16,0.0,...,2013-08-17,0.0,0.0,0.0,0.0,0.0,0.0,0.10,0.0,2017
499630,DHJxd5kJLlnhSJ/3W64eUCNKq4e6sHW/Mkb5cTKgqtU=,0,41.0,30.0,99.0,99.0,1.0,2017-03-02,2017-04-02,0.0,...,2013-09-06,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,2017
499757,GFLZXczzbNX7GoFp59rVb7cahr2wgShXBbxO/zZM13c=,0,36.0,30.0,180.0,180.0,1.0,2017-03-23,2017-04-22,0.0,...,2013-11-20,1.0,0.0,0.0,0.0,0.0,0.0,0.03,0.0,2017


In [26]:
#month
train_df['last_transaction_month'] = train_df['transaction_date'].dt.month
train_df = encode(train_df, 'last_transaction_month', 12)
train_df.head(2)

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,...,period_-2_churn,period_-3_churn,period_-4_churn,period_-5_churn,usage_from_ltd,discount_percentage,last_transaction_year,last_transaction_month,last_transaction_month_sin,last_transaction_month_cos
40,WpYNkiRMP3vgJRyq4TQzn7y8O0I7RWtXLXUorHClllg=,1,39.0,30.0,149.0,149.0,1.0,2017-02-28,2017-04-06,0.0,...,0.0,0.0,0.0,0.0,0.01,0.0,2017,2,0.866025,0.5
41,2OaV7/7vyAA2FrCkV4qXZEZYg5IzVyFNkQnQorgQ440=,1,41.0,30.0,149.0,149.0,1.0,2017-02-28,2017-06-08,0.0,...,1.0,0.0,0.0,0.0,0.03,0.0,2017,2,0.866025,0.5


In [27]:
#day
train_df['last_transaction_day'] = train_df['transaction_date'].dt.day
train_df= encode(train_df, 'last_transaction_day', 31)

In [28]:
train_df =train_df.drop(['last_transaction_month', 'last_transaction_day'],axis=1)

In [29]:
# cyclic encoding membership expire date
#year
train_df['expire_year'] = train_df['membership_expire_date'].dt.year

#month
train_df['expire_month'] = train_df['membership_expire_date'].dt.month
train_df = encode(train_df, 'expire_month', 12)

#day
train_df['expire_day'] = train_df['membership_expire_date'].dt.day
train_df= encode(train_df, 'expire_day', 31)

# droping month and day columns
train_df =train_df.drop(['expire_month', 'expire_day'],axis=1)

In [30]:
# cyclic encoding registration date
#year
train_df['registration_year'] = train_df['registration_init_time'].dt.year

#month
train_df['registration_month'] = train_df['registration_init_time'].dt.month
train_df = encode(train_df, 'registration_month', 12)

#day
train_df['registration_day'] = train_df['registration_init_time'].dt.day
train_df= encode(train_df, 'registration_day', 31)

# droping month and day columns
train_df =train_df.drop(['registration_month', 'registration_day'],axis=1)


In [31]:
# droping date columns to avoid data leakege
train_df =train_df.drop(['registration_init_time', 'membership_expire_date', 'transaction_date'],axis=1)

In [32]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2501 entries, 40 to 499914
Data columns (total 43 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   msno                        2501 non-null   object 
 1   is_churn                    2501 non-null   int64  
 2   payment_method_id           2501 non-null   float64
 3   payment_plan_days           2501 non-null   float64
 4   plan_list_price             2501 non-null   float64
 5   actual_amount_paid          2501 non-null   float64
 6   is_auto_renew               2501 non-null   float64
 7   is_cancel                   2501 non-null   float64
 8   remaining_plan_duration     2501 non-null   int64  
 9   is_discount                 2501 non-null   object 
 10  num_25                      2501 non-null   float64
 11  num_50                      2501 non-null   float64
 12  num_75                      2501 non-null   float64
 13  num_985                     2501 no

In [33]:
train_df.to_csv('churn1_df.csv')